In [18]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

## IMPORTING AND SETTING UP THE SEQUENCES

In [19]:
sequences = pd.read_csv('w.csv', sep='\t', header=0)

In [20]:
sequences

,hotels
0,"['1943423', '5603582', '584327']"
1,['2842610']
2,['4869510']
3,['2418979']
4,"['3311750', '4130544', '6162264']"
...,...
143389,"['3959122', '6437848', '4973328', '4182298', '..."
143390,"['2256352', '816462']"
143391,"['4807793', '1624053', '3634417', '4059241', '..."
143392,"['2145364', '221802']"


In [21]:
def func(x):
    x = x[1:-1]
    x = x.replace(" ", "")
    x = x.replace("'", "")
    x = x.split(',')
    return x

In [22]:
sequences = sequences.hotels.apply(func)

In [23]:
sequences

0                                [1943423, 5603582, 584327]
1                                                 [2842610]
2                                                 [4869510]
3                                                 [2418979]
4                               [3311750, 4130544, 6162264]
                                ...                        
143389    [3959122, 6437848, 4973328, 4182298, 335995, 3...
143390                                    [2256352, 816462]
143391    [4807793, 1624053, 3634417, 4059241, 5284416, ...
143392                                    [2145364, 221802]
143393                          [3284844, 1958933, 1317946]
Name: hotels, Length: 143394, dtype: object

## TRAIN TEST SPLIT

In [24]:
#1) Split train and test






from sklearn.model_selection import train_test_split



X_train, X_test = train_test_split(
  sequences,  test_size=0.2, random_state=42)

k = 3
for i in range(1,6):
    for j in range(200,2100,200):
        mymodel = Word2Vec(X_train,min_count = i,vector_size = 32,negative = j, workers = k)
        mymodel.save(f"hotel2vec,min{i}neg{j}work{k}window5.model")

In [25]:
X_train

93369     [2439858, 1000465, 353916, 4447057, 6902250, 1...
83933                     [2856315, 629844, 625780, 633558]
21737                                    [1127179, 1274101]
29341     [3325337, 4895761, 980408, 2019205, 1786106, 2...
6692                                     [5134132, 3362734]
                                ...                        
110268                 [1786200, 6518774, 3515800, 4861669]
119879                          [4214243, 1566354, 3221504]
103694                                   [4528588, 1622147]
131932    [1750386, 3402888, 33186, 3972860, 4077320, 66...
121958                          [2130028, 6677420, 3805991]
Name: hotels, Length: 114715, dtype: object

k = 3
for i in range(1,6):
    for j in range(200,2100,200):
        for l in range(10,50,10):
            mymodel = Word2Vec(X_train,min_count = i,vector_size = 32,negative = j,window = l, workers = k)
            mymodel.save(f"hotel2vec,min{i}neg{j}work{k}window{l}.model")

## LOAD MODELS

In [26]:
X_test = X_test.reset_index()
X_test.drop('index', inplace=True, axis=1)

### TRANSFORMING SEQUENCES TO X,Y

### MEAN OF PREVIOUS CLICKS

In [46]:
def meanclick(model,X_test):

    mymodel = KeyedVectors.load(model, mmap='r') 
    def erase_voc(X):
        k = X.copy()
        for i in X:
            if i not in mymodel.wv.index_to_key:
                k.remove(i)
        return k
    def tomodel(x):
        a=[]
        for i in x:
            a.append(mymodel.wv[i])
        return a 
    X_new = X_test.hotels.apply(erase_voc)
    X_new = X_new[X_new.apply(lambda x : len(x)> 1)]
    X_new1 = X_new.apply(tomodel)
    X_new = X_new.reset_index()
    X_new.drop('index', inplace=True, axis=1)
    X_new1 = X_new1.reset_index()
    X_new1.drop('index', inplace=True, axis=1)
    A = []
    B = []
    for i in range(0,len(X_new.hotels)):
        a = [X_new1.hotels[i][0]]
        x = [X_new1.hotels[i][0]]
        y = [X_new.hotels[i][1]]
        for j in range(1,len(X_new1.hotels[i])-1):

            a.append(X_new1.hotels[i][j])
            x.append(sum(a)/len(a))
            y.append(X_new.hotels[i][j+1])
        A.append(x)
        B.append(y)
    X_te = [i for sublist in A for i in sublist]
    Y_te = [i for sublist in B for i in sublist]  
    data = pd.DataFrame(Y_te,columns = ['Y_te'])
    data['X_te'] = X_te
    return data

### PREVIOUS CLICK

In [47]:
def previousclick(model,X_test):
        mymodel = KeyedVectors.load(model, mmap='r') 
        def erase_voc(X):
            k = X.copy()
            for i in X:
                if i not in mymodel.wv.index_to_key:
                    k.remove(i)
            return k
        def tomodel(x):
            a=[]
            for i in x:
                a.append(mymodel.wv[i])
            return a

                   
        X_new = X_test.hotels.apply(erase_voc)
        X_new = X_new[X_new.apply(lambda x : len(x)> 1)]
        X_new1 = X_new.apply(tomodel)
        X_new = X_new.reset_index()
        X_new.drop('index', inplace=True, axis=1)
        X_new1 = X_new1.reset_index()
        X_new1.drop('index', inplace=True, axis=1)
        A = []
        B = []
        for i in range(0,len(X_new.hotels)):
            
            x = [X_new1.hotels[i][0]]
            y = [X_new.hotels[i][1]]
            for j in range(1,len(X_new1.hotels[i])-1):
                
                x.append(X_new.hotels[i][j])
                y.append(X_new.hotels[i][j+1])
            A.append(x)
            B.append(y)
        X_te = [i for sublist in A for i in sublist]
        Y_te = [i for sublist in B for i in sublist]  
        data = pd.DataFrame(Y_te,columns = ['Y_te'])
        data['X_te'] = X_te
        return data

## EVALUATION METRICS

### HITS@K

In [110]:
def hits(dataframe,K,sample,model):
    mymodel = KeyedVectors.load(model, mmap='r') 
    def sim_hotels(vector,topn):
        
        a = []
        for i in range(0,len(mymodel.wv.similar_by_vector(vector, topn=topn, restrict_vocab=None))):
            a.append(mymodel.wv.similar_by_vector(vector, topn=topn,restrict_vocab=None)[i][0])
        return a
    if sample==1:
        dataframe = dataframe.sample(1000,random_state = 6)                 

    dataframe['hits'] = 0
    dataframe['hits'] = dataframe.apply(lambda row:  row.Y_te in sim_hotels(row.X_te,K),axis = 1);
    hits = dataframe.hits.value_counts(normalize = True)
    if len(hits) ==2:
        value = hits[1]
    else:
        value = 0
    return value

### MEAN RECIPROCAL RANK@K

In [135]:
def MRR(dataframe,K,sample,model):
    mymodel = KeyedVectors.load(model, mmap='r') 
    def sim_hotels(vector,topn):
        a = []
        for i in range(0,len(mymodel.wv.similar_by_vector(vector, topn=topn, restrict_vocab=None))):
            a.append(mymodel.wv.similar_by_vector(vector, topn=topn,restrict_vocab=None)[i][0])
        return a
    def rr(x,y):
            s = 0
            for i in range(0,len(y)):
                if x== y[i]:
                    s = 1/(i+1)
                    break
            return s
                  
    
    if sample==1:
        dataframe = dataframe.sample(1000,random_state = 6)    
    dataframe['hits'] = 0
    dataframe['hits'] = dataframe.apply(lambda row:  rr(row.Y_te,sim_hotels(row.X_te,K)),axis = 1);
    hits = dataframe.hits.value_counts(normalize = True)
    if len(hits) ==2:
        value = hits[1]
    else: value = 0
    return value
    return print('model: ',model,' hits@k: ',hits)

## MODEL SELECTION BASED ON METRICS

### MEAN OF PREVIOUS CLICKS

In [112]:
k = 3

evaluation_list1 = {}
for i in range(1,3): 
    for j in range(5,21,5):
    
            ten = hits(meanclick(f'hotel2vec,min{i}neg{j}work{k}skiphs.model',X_test),
                10, 1,  f'hotel2vec,min{i}neg{j}work{k}skiphs.model')
            evaluation_list1[f'negative = {j},words_min = {i}'] = ten
        
for j in range(40,101,20):
    for i in range(1,3):
        ten = hits(meanclick(f'hotel2vec,min{i}neg{j}work{k}skiphs.model',X_test),
                10, 1,  f'hotel2vec,min{i}neg{j}work{k}skiphs.model')
        evaluation_list1[f'negative = {j},words_min = {i}'] = ten
    
for j in range(200,1001,100):
    for i in range(1,3):
        ten = hits(meanclick(f'hotel2vec,min{i}neg{j}work{k}skiphs.model',X_test),
                10, 1,  f'hotel2vec,min{i}neg{j}work{k}skiphs.model')
        evaluation_list1[f'negative = {j},words_min = {i}'] = ten

In [113]:
evaluation_list_hits10_mean_of_clicks = evaluation_list1
evaluation_list_hits10_mean_of_clicks

{'negative = 5,words_min = 1': 0.118,
 'negative = 10,words_min = 1': 0.105,
 'negative = 15,words_min = 1': 0.097,
 'negative = 20,words_min = 1': 0.087,
 'negative = 5,words_min = 2': 0.142,
 'negative = 10,words_min = 2': 0.118,
 'negative = 15,words_min = 2': 0.126,
 'negative = 20,words_min = 2': 0.122,
 'negative = 40,words_min = 1': 0.086,
 'negative = 40,words_min = 2': 0.113,
 'negative = 60,words_min = 1': 0.086,
 'negative = 60,words_min = 2': 0.108,
 'negative = 80,words_min = 1': 0.089,
 'negative = 80,words_min = 2': 0.119,
 'negative = 100,words_min = 1': 0.084,
 'negative = 100,words_min = 2': 0.11,
 'negative = 200,words_min = 1': 0.073,
 'negative = 200,words_min = 2': 0.098,
 'negative = 300,words_min = 1': 0.063,
 'negative = 300,words_min = 2': 0.093,
 'negative = 400,words_min = 1': 0.067,
 'negative = 400,words_min = 2': 0.101,
 'negative = 500,words_min = 1': 0.065,
 'negative = 500,words_min = 2': 0.088,
 'negative = 600,words_min = 1': 0.063,
 'negative = 600,

In [121]:
import operator
best_case = max(evaluation_list_hits10_mean_of_clicks.items(), key=operator.itemgetter(1))
best_case

('negative = 5,words_min = 2', 0.142)

### PREVIOUS CLICK

In [114]:
k = 3

evaluation_list2 = {}

for i in range(1,3): 
    for j in range(5,21,5):
    
            ten = hits(previousclick(f'hotel2vec,min{i}neg{j}work{k}skiphs.model',X_test),
                10, 1,  f'hotel2vec,min{i}neg{j}work{k}skiphs.model')
            evaluation_list2[f'negative = {j},words_min = {i}'] = ten
        
for i in range(1,3):
    for j in range(40,101,20):

        ten = hits(previousclick(f'hotel2vec,min{i}neg{j}work{k}skiphs.model',X_test),
                10, 1,  f'hotel2vec,min{i}neg{j}work{k}skiphs.model')
        evaluation_list2[f'negative = {j},words_min = {i}'] = ten
    
for i in range(1,3):
    for j in range(200,1001,100):

        ten = hits(previousclick(f'hotel2vec,min{i}neg{j}work{k}skiphs.model',X_test),
                10, 1,  f'hotel2vec,min{i}neg{j}work{k}skiphs.model')
        evaluation_list2[f'negative = {j},words_min = {i}'] = ten

In [116]:
evaluation_list_hits10_previous_click = evaluation_list2
evaluation_list_hits10_previous_click

{'negative = 5,words_min = 1': 0.136,
 'negative = 10,words_min = 1': 0.14,
 'negative = 15,words_min = 1': 0.136,
 'negative = 20,words_min = 1': 0.148,
 'negative = 5,words_min = 2': 0.165,
 'negative = 10,words_min = 2': 0.176,
 'negative = 15,words_min = 2': 0.167,
 'negative = 20,words_min = 2': 0.166,
 'negative = 40,words_min = 1': 0.138,
 'negative = 60,words_min = 1': 0.122,
 'negative = 80,words_min = 1': 0.131,
 'negative = 100,words_min = 1': 0.124,
 'negative = 40,words_min = 2': 0.167,
 'negative = 60,words_min = 2': 0.15,
 'negative = 80,words_min = 2': 0.154,
 'negative = 100,words_min = 2': 0.149,
 'negative = 200,words_min = 1': 0.114,
 'negative = 300,words_min = 1': 0.101,
 'negative = 400,words_min = 1': 0.106,
 'negative = 500,words_min = 1': 0.091,
 'negative = 600,words_min = 1': 0.082,
 'negative = 700,words_min = 1': 0.084,
 'negative = 800,words_min = 1': 0.074,
 'negative = 900,words_min = 1': 0.081,
 'negative = 1000,words_min = 1': 0.079,
 'negative = 200,

In [122]:
import operator
best_case = max(evaluation_list_hits10_previous_click.items(), key=operator.itemgetter(1))
best_case

('negative = 10,words_min = 2', 0.176)

According to the above hits@10 metrics of the sample we choose model with negative sample equal to 5 and word minimum of 2 and
the aggregation which only includes the previous click.
Below we will compute the complete hits@10,hits@100, mrr@10 and mrr@100.

### FINAL MODEL METRICS

In [ ]:
model_evaluation = {}
model_evaluation['hits@10'] = hits(previousclick('hotel2vec,min2neg5work3skiphs.model',X_test),
        10, 0,  'hotel2vec,min2neg5work3skiphs.model')
model_evaluation['hits@100'] = hits(previousclick('hotel2vec,min2neg5work3skiphs.model',X_test),
        100, 0,  'hotel2vec,min2neg5work3skiphs.model')


In [137]:
model_evaluation

{'hits@10': 0.15704293786666346, 'hits@100': 0.5288471920626732, 'MRR@10': 0}

In [ ]:
model_evaluation['MRR@10'] = MRR(previousclick('hotel2vec,min2neg5work3skiphs.model',X_test),
        10, 0,  'hotel2vec,min2neg5work3skiphs.model')

model_evaluation['MRR@100'] = MRR(previousclick('hotel2vec,min2neg5work3skiphs.model',X_test),
        100, 0,  'hotel2vec,min2neg5work3skiphs.model')